In [2]:
!pip install livelossplot

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, EarlyStopping
from tensorflow.keras.utils import plot_model

"""
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_normal
"""
from IPython.display import SVG, Image
from livelossplot import PlotLossesKerasTF
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.1.0


In [5]:
train_data_dir="/kaggle/input/courserafacemoreco/train/"
validation_data_dir="/kaggle/input/courserafacemoreco/test/"

In [6]:
num_classes = 0
for expression in os.listdir(train_data_dir):
    print(str(len(os.listdir(train_data_dir+expression)))+" "+expression +" images")
    num_classes += 1
print(num_classes)

3171 surprise images
3995 angry images
7214 happy images
4830 sad images
4965 neutral images
5


In [7]:
img_rows, img_cols = 224, 224
# img_size = 48 # old
img_size = 224 
batch_size = 64 # or 32



# datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True) # old
datagen_train = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=30,
                                   width_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   height_shift_range=0.3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = datagen_train.flow_from_directory(train_data_dir,
                                                    target_size=(img_rows,img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

# datagen_validation = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True) # old
datagen_validation = ImageDataGenerator(rescale=1. / 255,horizontal_flip=True)

validation_generator = datagen_validation.flow_from_directory(validation_data_dir,
                                                              target_size=(img_rows,img_cols),
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              shuffle=False)

Found 24175 images belonging to 5 classes.
Found 6043 images belonging to 5 classes.


In [8]:
print(train_generator.n)
print(validation_generator.n)

24175
6043


In [13]:
IMG_SHAPE = (img_rows, img_cols, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [14]:
base_model.trainable = True

In [15]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [16]:
top_model = base_model.output
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
top_model = global_average_layer(top_model)
print(top_model.shape)

(None, 1280)


In [17]:
top_model = tf.keras.layers.Dense(num_classes, activation='softmax')(top_model)

#model = tf.keras.Sequential([base_model,global_average_layer,top_model])
model = tf.keras.models.Model(inputs=base_model.input, outputs=top_model)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________________

In [19]:
opt = tf.keras.optimizers.Adam(lr=0.0005)
loss_categ = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_categ, metrics=['accuracy'])

In [21]:
epochs = 200

steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=5,
                              min_lr=0.00001,
                              verbose=1, #me
                              mode='auto')


checkpoint = ModelCheckpoint("CNNMobileNetV2EmoDetAllTrainable_weights.h5",
                             monitor='val_loss',
                             save_best_only=True,#me
                             save_weights_only=True,
                             mode='min',
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=14,
                          verbose=1,
                          restore_best_weights=True)


callbacks = [PlotLossesKerasTF(), checkpoint, reduce_lr, earlystop]

In [ ]:
history = model.fit(x=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps,
                    callbacks=callbacks)

Train for 377 steps, validate for 94 steps
Epoch 1/200
 11/377 [..............................] - ETA: 52:03 - loss: 1.7200 - accuracy: 0.2557

In [ ]:
plot_model(model, to_file='CNNMobileNetV2EmoDetAllTrainable.png', show_shapes=True)

In [ ]:
model.save("CNNMobileNetV2EmoDetAllTrainable.h5")

In [ ]:
model_json = model.to_json()
with open("JCNNMobileNetV2EmoDetAllTrainable.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()